In [2]:
import pandas as pd
import numpy as np

In [3]:
#read in tagged data
instance_dataset_path = "src/instances_Robuddy_with_tag.csv"
df = pd.read_csv(instance_dataset_path)
instance_cnt = len(df)

In [4]:
# save instance freq
import os
import pickle
# if instance freq not exists
if not os.path.exists("statistics/instance_freq.pkl"):
    instance_freq = np.zeros(instance_cnt,dtype=np.int32)
    with open("statistics/instance_freq.pkl",'wb') as f:
        pickle.dump(instance_freq,f)
else:
    with open("statistics/instance_freq.pkl",'rb') as f:
        instance_freq = pickle.load(f)

In [ ]:
# load the ability-index dict
import json

ability_index_dict = {}
ability_json_file = "src/ability_mindmap.json"
with open(ability_json_file,'r') as f:
    ability_tree = json.load(f)

index = 0   

def traverse_ability_tree(node):
    ability_index_dict[node["name"]] = index
    index += 1
    if "children" in node:
        for child in node["children"]:
            traverse_ability_tree(child)

traverse_ability_tree(ability_tree)

In [ ]:
# init ability data
if os.path.exists("statistics/ability_pair_ref.pickle"):
    with open("statistics/ability_pair_ref.pickle",'rb') as f:
        ability_pair_ref = pickle.load(f)
else:
    # first calculate the tagged ability appearances in raw df
    ability_freq = {} 
    for i in range(len(df)):
        abilities = df.iloc[i]["annotation"]
        ability_list = abilities.split(",")
        for ability in ability_list:
            if ability not in ability_freq:
                ability_freq[ability] = 0
            ability_freq[ability] += 1
        
    def add_freq_to_parent(node):
        if "children" in node:
            for child in node["children"]:
                add_freq_to_parent(child)
                ability_freq[node["name"]] = ability_freq.get(node["name"],0) + ability_freq.get(child["name"],0)
        else:
            ability_freq[node["name"]] = ability_freq.get(node["name"],0)
    
    add_freq_to_parent(ability_tree)

    # then calculate the ability pair frequency
    ability_pair_ref = np.zeros((len(ability_index_dict),len(ability_index_dict)),dtype=np.float32)
    for ability,index in ability_index_dict.items():
        for ability_2,index_2 in ability_index_dict.items():
            ability_pair_ref[index][index_2] = ability*ability_2
    # normalize
    ability_pair_freq = ability_pair_ref/np.sum(ability_pair_ref)
    with open("statistics/ability_pair_ref.pickle",'wb') as f:
        pickle.dump(ability_pair_ref,f)

In [5]:
#  load ability pair data:
if os.path.exists("statistics/ability_pair_freq.pickle"):
    with open("statistics/ability_pair_freq.pickle",'rb') as f:
        ability_pair_freq = pickle.load(f)
else:
    ability_pair_freq = np.zeros((len(ability_index_dict),len(ability_index_dict)),dtype=np.int32)
    with open("statistics/ability_pair_freq.pickle",'wb') as f:
        pickle.dump(ability_pair_freq,f)